##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow による分散型トレーニング

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/distributed_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

`tf.distribute.Strategy` は、複数の GPU、複数のマシン、または TPU でトレーニングを分散する TensorFlow API です。この API を使用すると、最小限のコード変更により、既存のモデルとトレーニングコードを分散することができます。

`tf.distribute.Strategy` は、次の主な目標を念頭に設計されています。

- 使いやすく、研究者や ML エンジニアなど、複数のユーザーセグメントをサポートすること。
- 調整することなく優れたパフォーマンスを提供できること。
- ストラテジーを簡単に切り替えられること。

You can distribute training using `tf.distribute.Strategy` with a high-level API like Keras `Model.fit`, as well as [custom training loops](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) (and, in general, any computation using TensorFlow).

TensorFlow 2.x では、プログラムを Eager モードで、または [`tf.function`](function.ipynb) を使用してグラフ内で実行することができます。`tf.distribute.Strategy` はこれらの実行モードをサポートするのが狙いですが、`tf.function` を使用するのが最適です。Eager モードはデバッグ目的にのみ推奨され、`TPUStrategy` ではサポートされていません。このガイドの大半はトレーニングに関する内容ですが、異なるプラットフォームに評価と予測を分散する上でも、この API を使用することができます。

`tf.distribute.Strategy` は、コードをほとんど変更せずに使用することができます。TensorFlow の基盤のコンポーネントをストラテジーを認識するように変更されているからです。これには、変数、レイヤー、モデル、オプティマイザー、メトリック、要約、およびチェックポイントが含まれます。

このガイドでは、さまざまなストラテジーと、異なる状況においてそれらのストラテジーをどのように使用するかについて説明します。

注意: 概念の理解を深めるには、[こちらの詳しいプレゼンテーション](https://youtu.be/jKV53r9-H14)をご覧ください。独自のトレーニングループを記述する予定の方に、特にお勧めです。


## Set up TensorFlow

In [ ]:
import tensorflow as tf

## ストラテジーの種類

`tf.distribute.Strategy` は、さまざまな軸に沿って多数の使用事例をカバーすることを目的としています。これらの組み合わせの中には、現在サポートされているものもあれば、今後の追加が予定されているものもあります。次に一部の軸を示します。

- *同期と非同期トレーニング:* これらは、データの並列処理を使用してトレーニングを分散する 2 つの一般的な方法です。同期トレーニングでは、すべてのワーカーは入力データのさまざまなスライスで同期的にトレーニングし、各ステップで勾配を収集します。一方、非同期トレーニングでは、すべてのワーカーは独立的に入力データでトレーニングし、変数を非同期的に更新します。通常、同期トレーニングは all-reduce を介して、非同期トレーニングはパラメーターサーバーアーキテクチャを介してサポートされています。
- *ハードウェアプラットフォーム:* トレーニングを、1台のマシンの複数の GPU またはネットワーク内の複数のマシン（それぞれに 0 個以上の GPU）、さらには Cloud TPU に拡張することができます。

こういった使用事例をサポートするために、6 つのストラテジーが用意されています。次のセクションにおいて、現時点の TF 2.2 においてどれがどのシナリオをサポートしているのかについて説明しますが、以下に要約を示します。

トレーニング API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `CentralStorageStrategy` | `ParameterServerStrategy`
:-- | :-- | :-- | :-- | :-- | :--
**Keras API** | サポート中 | サポート中 | 実験的サポート | 実験的サポート | 2.3 以降でサポート予定
**カスタムトレーニングループ** | サポート中 | サポート中 | 実験的サポート | 実験的サポート | 2.3 以降でサポート予定
**Estimator API** | 制限サポート | 未サポート | 制限サポート | 制限サポート | 制限サポート

注意: 「[実験的サポート](https://www.tensorflow.org/guide/versions#what_is_not_covered)」とは、API の互換性が保証されていないことを意味します。

注意: Estimator のサポートは制限されています。基本的なトレーニングと評価は実験的であり、scaffold といった高度な機能は実装されていません。使用事例がカバーされていない場合は、Keras やカスタムトレーニングループの使用をお勧めします。

### MirroredStrategy

`tf.distribute.MirroredStrategy` は、1 台のマシンの複数の GPU での同期分散型トレーニングをサポートしています。GPU デバイス当たり 1 つのレプリカを作成します。モデルの各変数はそのレプリカ全体にミラーリングされます。これらの変数を合わせて形成されるのが、`MirroredVariable` と呼ばれる 1 つの概念的変数です。これらの変数は、同一の更新を適用することで、相互の同期が維持されます。

Efficient all-reduce algorithms are used to communicate the variable updates across the devices. All-reduce aggregates tensors across all the devices by adding them up, and makes them available on each device. It’s a fused algorithm that is very efficient and can reduce the overhead of synchronization significantly. There are many all-reduce algorithms and implementations available, depending on the type of communication available between devices. By default, it uses the NVIDIA Collective Communication Library ([NCCL](https://developer.nvidia.com/nccl)) as the all-reduce implementation. You can choose from a few other options or write your own.

次は、最も単純な `MirroredStrategy` の作成方法です。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

これにより、TensorFlow が認識できるすべての GPU を使用する `MirroredStrategy` インスタンスが作成され、クロスデバイス通信として NCCL が使用されるようになります。

マシン上の一部の GPU のみを使用する場合は、次のように行うことができます。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

クロスデバイス通信をオーバーライドする場合は、`cross_device_ops` 引数に `tf.distribute.CrossDeviceOps` のインスタンスを提供することで実行できます。現在のところ、デフォルトの `tf.distribute.NcclAllReduce` のほかに、`tf.distribute.HierarchicalCopyAllReduce` と `tf.distribute.ReductionToOneDevice` の 2 つのオプションを使用できます。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

### TPUStrategy

`tf.distribute.experimental.TPUStrategy` は、Tensor Processing Unit（TPU）で TensorFlow トレーニングを実行できるようにします。TPU は、機械学習ワークロードを劇的に高速化するように設計された Google の特殊 ASIC です。Google Colab の [TensorFlow Research Cloud](https://www.tensorflow.org/tfrc) と [Cloud TPU](https://cloud.google.com/tpu) で利用できます。

分散型トレーニングアーキテクチャの観点で言えば、`TPUStrategy` は `MirroredStrategy` と同じで、同期分散型トレーニングを実装します。TPU は複数の TPU コアに、効率的な all-reduce の独自の実装とほかの集合演算を提供しており、`TPUStrategy` で使用されています。

以下に、`TPUStrategy` をインスタンス化する方法を示します。

注意: Colab でこのコードを実行するには、Colab ランタイムとして TPU を選択する必要があります。[TensorFlow TPU ガイド](https://www.tensorflow.org/guide/tpu)を参照してください。

```python
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(     tpu=tpu_address) tf.config.experimental_connect_to_cluster(cluster_resolver) tf.tpu.experimental.initialize_tpu_system(cluster_resolver) tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
```

`TPUClusterResolver` インスタンスを使って TPU の場所を特定することができます。Colab では、引数を指定する必要はありません。

Cloud TPU でこれを使用する場合は、次の内容に注意してくだい。

- `tpu` 引数に、TPU リソースの名前を指定する必要があります。
- You must initialize the TPU system explicitly at the *start* of the program. This is required before TPUs can be used for computation. Initializing the TPU system also wipes out the TPU memory, so it's important to complete this step first in order to avoid losing state.

### MultiWorkerMirroredStrategy

`tf.distribute.experimental.MultiWorkerMirroredStrategy` は、`MirroredStrategy` に非常に似通ったストラテジーです。複数のワーカーにそれぞれ潜在的に複数の GPU を使って同期分散型トレーニングを実装します。`MirroredStrategy` と同様に、すべてのワーカーの各デバイスのモデルにすべての変数のコピーを作成します。

マルチワーカー all-reduce 通信方法として [CollectiveOps](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/collective_ops.py) を使用して、変数の同期を維持しています。collective 演算は、TensorFlow グラフの単一の演算で、ハードウェア、ネットワークテクノロジー、およびテンソルのサイズに応じて TensorFlow ランタイムに all-reduce アルゴリズムを自動的に選択することができます。

In [ ]:
multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

`MultiWorkerMirroredStrategy` では、現在、2 つの collective 演算の実装を選択することができます。 `CollectiveCommunication.RING` は、gRPC を使用してリング型の collective を通信レイヤーとして実装します。 `CollectiveCommunication.NCCL` は、[Nvidia の NCCL](https://developer.nvidia.com/nccl) を使用して集合を実装します。`CollectiveCommunication.AUTO` は、選択をランタイムに任せます。collective 実装の最適な選択は、GPU の数と種類、およびクラスタ内のネットワーク相互接続によって決まります。これらは、次のようにして指定することができます。


In [ ]:
multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    tf.distribute.experimental.CollectiveCommunication.NCCL)

マルチワーカートレーニングを進める上でマルチ GPU トレーニングとの主な違いの 1 つに、マルチワーカーのセットアップが挙げられます。`TF_CONFIG` 環境変数は、TensorFlow においてクラスタ構成をそのクラスタの一部である各ワーカーに指定する標準的な方法です。[TF_CONFIG の設定](#TF_CONFIG)についてさらにお読みください。

For more details about `MultiWorkerMirroredStrategy`, consider the following tutorials:

- [Multi-worker training with Keras Model.fit](../tutorials/distribute/multi_worker_with_keras.ipynb)
- [Multi-worker training with a custom training loop](../tutorials/distribute/multi_worker_with_ctl.ipynb)

### ParameterServerStrategy

`tf.distribute.experimental.ParameterServerStrategy` は、複数のマシンでのパラメーターサーバートレーニングをサポートします。このセットアップでは、マシンはワーカーとパラメーターサーバーのいずれかに指定されます。モデルの各変数は、1 つのパラメーターサーバーに配置されます。計算は、すべてのワーカーの全 GPU で複製されます。

コードは、ほかのストラテジーに似通っています。

In this implementation, the `worker` and `parameter server` tasks run `tf.distribute.Server`s that listen for tasks from the coordinator. The coordinator creates resources, dispatches training tasks, writes checkpoints, and deals with task failures.

In the programming running on the coordinator, you will use a `ParameterServerStrategy` object to define a training step and use a `ClusterCoordinator` to dispatch training steps to remote workers. Here is the simplest way to create them:

```python
ps_strategy = tf.distribute.experimental.ParameterServerStrategy()
```

To learn more about `ParameterServerStrategy`, check out the [Parameter server training with Keras Model.fit and a custom training loop](../tutorials/distribute/parameter_server_training.ipynb) tutorial.

Note: You will need to configure the `'TF_CONFIG'` environment variable if you use `TFConfigClusterResolver`. It is similar to <a href="#TF_CONFIG" data-md-type="link">`'TF_CONFIG'`</a> in `MultiWorkerMirroredStrategy` but has additional caveats.

In TensorFlow 1, `ParameterServerStrategy` is available only with an Estimator via `tf.compat.v1.distribute.experimental.ParameterServerStrategy` symbol.

注意: このストラテジーは現在改善中であり、より多くのシナリオで機能できるようにしていることから、「[`実験的`](https://www.tensorflow.org/guide/versions#what_is_not_covered)」に指定されています。この改善プロセスの一環として、API の振る舞いが今後変更される可能性があることにご注意ください。

### CentralStorageStrategy

`tf.distribute.experimental.CentralStorageStrategy` も同期トレーニングを行います。変数はミラーリングされず、代わりに CPU に配置され、演算はすべてのローカル GPU に複製されます。GPU が 1 つしかない場合は、すべての変数と演算はその GPU に配置されます。

次のようにして、`CentralStorageStrategy` のインスタンスを作成します。


In [ ]:
central_storage_strategy = tf.distribute.experimental.CentralStorageStrategy()

これにより、すべての GPU と CPU を使用する `CentralStorageStrategy` インスタンスが作成されます。レプリカでの変数の更新は、変数に適用される前に集計されます。

注意: このストラテジーは現在改善中であり、より多くのシナリオで機能できるようにしていることから、「[`実験的`](https://www.tensorflow.org/guide/versions#what_is_not_covered)」に指定されています。この改善プロセスの一環として、API の振る舞いが今後変更される可能性があることにご注意ください。

### その他のストラテジー

上述のストラテジーのほかに、`tf.distribute` API を使用する際のプロトタイピングとデバッグに役立つ可能性のあるストラテジーが 2 つあります。

#### デフォルトストラテジー

明示的な分散ストラテジーがスコープにない場合には、分散ストラテジーとしてデフォルトストラテジーが使用されます。`tf.distribute.Strategy` インターフェースを実装しますが、パススルーであるため、実際の分散を提供しません。たとえば、`strategy.run(fn)` は `fn` だけを呼び出します。このストラテジーを使用して書かれたコードは、ストラテジーを指定せずに書かれたコードとまった同じ振る舞いとなります。「演算なし」ストラテジーとして考えるとわかりやすいかもしれません。

デフォルトストラテジーはシングルトンであり、インスタンスを 1 つしか作成できません。明示的なストラテジーのスコープの外で、`tf.distribute.get_strategy()` を使って取得することができます（明示的なストラテジーのスコープ内で現在のストラテジーを取得するために使用するのと同じ API）。

In [ ]:
default_strategy = tf.distribute.get_strategy()

このストラテジーには、主に 2 つの目的があります。

- 無条件で分散対応のライブラリコードを記述すること。たとえば、オプティマイザーにおいて、`tf.distribute.get_strategy()` を実行し、勾配を減らすためにそのストラテジーを使用します。常に、reduce API を呼び出せるストラテジーオブジェクトを返します。


In [ ]:
# In optimizer or other library code
# Get currently active strategy
strategy = tf.distribute.get_strategy()
strategy.reduce("SUM", 1.)  # reduce some values

- ライブラリコードと同様に、条件論理を必要とせずに、分散ストラテジーの有無に関係なく機能するエンドユーザーのプログラムを記述すること。これを説明したサンプルコードスニペットを、次に示します。

In [ ]:
if tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))

#### OneDeviceStrategy

`tf.distribute.OneDeviceStrategy` は、すべての変数と計算を単一の指定デバイスに配置するストラテジーです。

```python
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
```

このストラテジーは、多数の点において、デフォルトストラテジーと異なります。デフォルトストラテジーでは、分散ストラテジーを使用せずに TensorFlow を実行した場合と比べ、変数の配置ロジックに変化はありません。しかし、`OneDeviceStrategy` を使用した場合、そのスコープで作成されるすべての変数は、指定されたデバイスに明示的に配置されます。さらに、`OneDeviceStrategy.run` から呼び出される関数も、その指定デバイスに配置されるようになります。

このストラテジーを通じて分散された入力は、指定デバイスにプリフェッチされます。デフォルトストラテジーには、入力の分散はありません。

デフォルトストラテジーと同様に、このストラテジーも、複数のデバイス/マシンに実際に分散する別のストラテジーに切り替える前のコードテストに使用することができます。これにより、分散ストラテジーの仕組みはある程度はデフォルトストラテジーよりもが強化されますが、`MirroredStrategy` や `TPUStrategy` などを使用した場合ほど最大限には強化されません。ストラテジーが指定されていないかのようなコードの振る舞いを希望する場合は、デフォルトストラテジーを使用してください。

これまで、利用できるさまざまなストラテジーとそれらをインスタンス化する方法について説明してきました。次のいくつかのセクションでは、トレーニングを分散する上でストラテジーを使用するためのさまざまな方法について説明します。このガイドでは短いコードスニペットを示し、エンドツーエンドで実行できるチュートリアル全文にリンクしています。

## `tf.keras.Model.fit` を使った `tf.distribute.Strategy` の使用

TensorFlow の [Keras API 仕様](https://keras.io)の実装である `tf.keras` に `tf.distribute.Strategy` を統合しました。`tf.keras` は、モデルを構築してトレーニングするための高位 API です。`tf.keras` バックエンドに統合することによって、`model.fit` を使用する Keras トレーニングフレームワークに記述されたトレーニングの分散をシームレスに行えるようになっています。

コードを次のように変更してください。

1. 適切な `tf.distribute.Strategy` のインスタンスを作成します。
2. Keras モデル、オプティマイザー、およびメトリックの作成を `strategy.scope` 内に移動します。

TensorFlow 分散ストラテジーは、Sequential、Functional、および Subclassed のすべての Keras モデルをサポートしています。

次に、1 つの高密度レイヤーを持つ非常に単純な Keras モデルに対してこれを実行するコードスニペットを示します。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
  model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])

model.compile(loss='mse', optimizer='sgd')

この例では、複数の GPU を持つマシンで実行できるように、`MirroredStrategy` を使用しています。`strategy.scope()` は、トレーニングの分散に度のストラテジーを使用するかを Keras に示しています。このスコープ内にモデル/オプティマイザー/メトリックを作成することで、通常の変数ではなく、分散化された変数を作成することができます。これをセットアップしたら、通常通り、モデルをフィットさせることができます。利用できる GPU に対するモデルのトレーニングの複製や勾配の集計などは、`MirroredStrategy` によって行われます。

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(10)
model.fit(dataset, epochs=2)
model.evaluate(dataset)

ここでは、`tf.data.Dataset` を使用してトレーニングと eval 入力を提供していますが、次のように numpy 配列を使用することもできます。

In [ ]:
import numpy as np
inputs, targets = np.ones((100, 1)), np.ones((100, 1))
model.fit(inputs, targets, epochs=2, batch_size=10)

いずれの場合（`Dataset` または Numpy）でも、指定された入力の各バッチは、複数のレプリカで均等に分割されます。たとえば、2 GPU で `MirroredStrategy` を使用している場合、サイズが 10 の各バッチは 2 GPU 間で分割されるため、各ステップでそれぞれに 5 つの入力例が提供されます。その後、GPU をさらに追加するにつれ、各エポックはより高速にトレーニングするようになります。一般的に、アクセラレータを増やすたびにバッチサイズを増加することが望ましく、そうすることで、追加の計算パワーを有効活用できるようになります。また、モデルに応じて学習速度を再調整することも必要です。レプリカの数を取得するには、`strategy.num_replicas_in_sync` を使用できます。

In [ ]:
# Compute global batch size using number of replicas.
BATCH_SIZE_PER_REPLICA = 5
global_batch_size = (BATCH_SIZE_PER_REPLICA *
                     mirrored_strategy.num_replicas_in_sync)
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100)
dataset = dataset.batch(global_batch_size)

LEARNING_RATES_BY_BATCH_SIZE = {5: 0.1, 10: 0.15}
learning_rate = LEARNING_RATES_BY_BATCH_SIZE[global_batch_size]

### 現在、何がサポートされていますか？

トレーニング API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `ParameterServerStrategy` | `CentralStorageStrategy`
--- | --- | --- | --- | --- | ---
Keras API | サポート中 | サポート中 | 実験的サポート | 実験的サポート | 2.3 以降でサポート予定

### 例とチュートリアル

次は、上述した Keras のエンドツーエンド統合を説明するチュートリアルと例の一覧です。

1. <code>MirroredStrategy</code> で MNIST をトレーニングする<a>チュートリアル</a>
2. `MultiWorkerMirroredStrategy` を使って MNIST をトレーニングする[チュートリアル](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras)
3. `TPUStrategy` による MNIST のトレーニングに関する[ガイド](https://www.tensorflow.org/guide/tpu#train_a_model_using_keras_high_level_apis)
4. さまざまなストラテジーを使って実装された最新モデルのコレクションが含まれる TensorFlow Model Garden [リポジトリ](https://github.com/tensorflow/models/tree/master/official)
5. [Tutorial](https://www.tensorflow.org/text/tutorials/bert_glue): Fine-tuning BERT for many tasks from the GLUE benchmark with `Model.fit` and `TPUStrategy`.
6. さまざまなストラテジーを使って実装された最新モデルのコレクションが含まれる TensorFlow Model Garden [リポジトリ](https://github.com/tensorflow/models/tree/master/official)

## カスタムトレーニングループを使った `tf.distribute.Strategy` の使用

これまで見てきたように、Keras の `model.fit` で `tf.distribute.Strategy` を使用するにはコードの数行のみを変更する必要がありました。もう少し手を加えるれば、カスタムトレーニングループでも `tf.distribute.Strategy` を使用できるようになります。

トレーニングループに Estimator や Keras よりも高い柔軟性と制御が必要な場合は、カスタムトレーニングループを作成することができます。たとえば、GANを使用する際に、各ラウンドで異なる数のジェネレータやディスクリミネータのステップを実行することができます。同様に、高度なフレームワークは、強化学習トレーニングにはあまり適していません。

カスタムとレニングループをサポートするために、`tf.distribute.Strategy` クラスでメソッドのコアセットを提供しています。これらを使用するには、最初にコードをわずかに再構成する必要があるかもしれませんが、それを一度行うと、ストラテジーインスタンスを変更するだけで、GPU、TPU、および複数のマシンを切り替えられるようになります。

ここでは、以前と同じ Keras モデルを使って、単純なトレーニングでの使用事例を表した簡単なスニペットを示します。


まず、ストラテジーのスコープ内にモデルとオプティマイザーを作成します。こうすることで、そのモデルとオプティマイザーを使って作成された変数がミラーリングされた変数であることを保証することができます。

In [ ]:
with mirrored_strategy.scope():
  model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
  optimizer = tf.keras.optimizers.SGD()

次に、入力データセットを作成し、`tf.distribute.Strategy.experimental_distribute_dataset` を呼び出して、ストラテジーに応じてデータセットを分散します。

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(
    global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

そして、トレーニングの一ステップを定義します。勾配とオプティマイザーを計算し、その勾配を適用してモデルの変数を更新するために、`tf.GradientTape` を使用します。このトレーニングステップを分散するには、`train_step` 関数に入れて、前に作成した `dist_dataset` から取得するデータセット入力とともに、`tf.distrbute.Strategy.run` に渡します。

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(
  from_logits=True,
  reduction=tf.keras.losses.Reduction.NONE)

def compute_loss(labels, predictions):
  per_example_loss = loss_object(labels, predictions)
  return tf.nn.compute_average_loss(per_example_loss, global_batch_size=global_batch_size)

def train_step(inputs):
  features, labels = inputs

  with tf.GradientTape() as tape:
    predictions = model(features, training=True)
    loss = compute_loss(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

@tf.function
def distributed_train_step(dist_inputs):
  per_replica_losses = mirrored_strategy.run(train_step, args=(dist_inputs,))
  return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

上記のコードには、注意すべき点がいくつかあります。

1. `tf.nn.compute_average_loss` を使用して損失を計算しました。`tf.nn.compute_average_loss` は例ごとの損失を加算し、その和を global_batch_size で除算します。これは、各レプリカで勾配が計算された後、その勾配を**加算**することで、レプリカ全体の勾配が集計されるため重要です。
2. `tf.distribute.Strategy.run` が返す結果を集計するために、`tf.distribute.Strategy.reduce` API を使用しました。`tf.distribute.Strategy.run` は、ストラテジーの各ローカルレプリカの結果を返し、この結果の消費方法もさまざまです。集計された値を取得するには、結果を `reduce` することができます。また、`tf.distribute.Strategy.experimental_local_results` を実行して、結果に含まれる、ローカルレプリカ当たり 1 つの値のリストを取得することもできます。
3. 分散ストラテジーのスコープ内で `apply_gradients` が呼び出されると、その振る舞いが変更されます。具体的には、同期トレーニング中に各並列インスタンスに勾配を適用する前に、勾配の sum-over-all-replicas を実行します。


最後に、トレーニングステップの定義が済んだら、`dist_dataset` をイテレートしてトレーニングをループで実行できます。

In [ ]:
for dist_inputs in dist_dataset:
  print(distributed_train_step(dist_inputs))

上記の例では、`dist_dataset` をイテレートして、トレーニングに入力を提供しました。また、numpy 入力をサポートするために、`tf.distribute.Strategy.make_experimental_numpy_dataset` も提供しています。`tf.distribute.Strategy.experimental_distribute_dataset` を呼び出す前にこの API を使って、データセットを作成することができます。

データをイテレートするための別の方法は、明示的にイテレータを使用することです。データセット全体をイテレートするのに対し、特定のステップ数だけ実行する場合に、これを行うことができます。上記のイテレートは、最初にイテレータを作成し、それに明示的に`next` を呼び出して入力データを取得するように変更されます。

In [ ]:
iterator = iter(dist_dataset)
for _ in range(10):
  print(distributed_train_step(next(iterator)))

これは、`tf.distribute.Strategy` APIを使用してカスタムトレーニングループを分散する最も単純な事例をカバーしています。これらの API は改善プロセスにあります。この使用事例にはコードを適合させる作業がさらに必要であるため、いずれ、詳細なガイドを別途公開する予定です。

### 現在、何がサポートされていますか？

トレーニング API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `ParameterServerStrategy` | `CentralStorageStrategy`
:-- | :-- | :-- | :-- | :-- | :--
カスタムトレーニングループ | サポート中 | サポート中 | 実験的サポート | 実験的サポート | 2.3 以降でサポート予定

### 例とチュートリアル

次は、カスタムトレーニングループを使って分散ストラテジーを使用するいくつかの例です。

1. `MirroredStrategy` で MNIST をトレーニングする[チュートリアル](https://www.tensorflow.org/tutorials/distribute/custom_training)
2. `TPUStrategy` による MNIST のトレーニングに関する[ガイド](https://www.tensorflow.org/guide/tpu#train_a_model_using_custom_training_loop)
3. さまざまなストラテジーを使って実装された最新モデルのコレクションが含まれる TensorFlow Model Garden [リポジトリ](https://github.com/tensorflow/models/tree/master/official)
4. [Tutorial](../tutorials/distribute/parameter_server_training.ipynb): Parameter server training with a custom training loop and `ParameterServerStrategy`.
5. さまざまなストラテジーを使って実装された最新モデルのコレクションが含まれる TensorFlow Model Garden [リポジトリ](https://github.com/tensorflow/models/tree/master/official)


## その他のトピック

このセクションでは、複数の使用事例に関連のあるトピックをいくつかカバーしています。

<a name="TF_CONFIG"></a>

### TF_CONFIG 環境変数のセットアップ

マルチワーカートレーニングでは、前述のとおり、クラスタで実行されている各バイナリに対して ` TF_CONFIG` 環境変数を設定する必要があります。`TF_CONFIG` 環境変数は、クラスタを構成するタスク、そのアドレス、およびクラスタ内の各タスクのロールを指定する JSON 文字列です。[tensorflow/ecosystem](https://github.com/tensorflow/ecosystem) レポジトリには、トレーニングタスクの `TF_CONFIG` を設定する Kubernotes テンプレートを提供しています。

TF_CONFIG には、cluster と task の 2 つのコンポーネントがあります。 cluster は、トレーニングクラスタに関する情報を提供します。これは、ワーカーなどのさまざまなタイプのジョブで構成される辞書です。マルチワーカートレーニングでは、通常、一般的なワーカーの作業に加えて、チェックポイントの保存や TensorBoard のサマリーファイルの書き込みなど、ほかよりタスクを担うワーカーが 1 つあります。こういったワーカーは、「チーフ」ワーカーと呼ばれ、インデックス 0 のワーカーがチーフワーカーに指定されるようになっています（事実、これが、tf.distribute.Strategy が実装された理由です）。一方の task は現在のタスクの情報を提供します。最初のコンポーネント cluster はすべてのワーカーで同じものですが、2 つ目のコンポーネント task は各ワーカーによって異なり、そのワーカーの種類とインデックスを指定します。

- A cluster provides information about the training cluster, which is a dict consisting of different types of jobs such as workers. In multi-worker training, there is usually one worker that takes on a little more responsibility like saving checkpoint and writing summary file for TensorBoard in addition to what a regular worker does. Such worker is referred to as the "chief" worker, and it is customary that the worker with index `0` is appointed as the chief worker (in fact this is how `tf.distribute.Strategy` is implemented).
- A task on the other hand provides information about the current task. The first component cluster is the same for all workers, and the second component task is different on each worker and specifies the type and index of that worker.

次は、`TF_CONFIG` の一例です。

```python
os.environ["TF_CONFIG"] = json.dumps({     "cluster": {         "worker": ["host1:port", "host2:port", "host3:port"],         "ps": ["host4:port", "host5:port"]     },    "task": {"type": "worker", "index": 1} })
```


この `TF_CONFIG`は、クラスタ内に 3 つのワーカーと 2 つの ps タスク、およびそれらのホストとポートがあることを指定しています。 「task」の部分は、クラスタ worker 1（2番目のワーカー）内の現在のタスクのロールを指定します。 クラスタ内の有効なロールは、「chi-ef」、「worker」、「ps」、および「evaluator」です。`tf.distribute.experimental.ParameterServerStrategy `を使用する場合を除いて、「ps」ジョブはありません。

## 次のステップ

`tf.distribute.Strategy` は積極的な開発が進められています。ぜひお試しいただき、[GitHub 課題](https://github.com/tensorflow/tensorflow/issues/new) に皆さんの感想をお寄せください。